**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Flatten, Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [46]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
            
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass
    
    def decay_epsilon(self, decay):
        self.epsilon *= decay

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The __act__ function use an $\epsilon$-greedy strategy.  
That is to say that with probability $\epsilon$ the algorithm will choose randomly an action among all the available actions
and with probability 1-$\epsilon$ it will choose the best action (the action that maximizes the reward):

$$a_{t+1} = \arg\max\limits_{a \in A} r(s_t, a_t)$$

By doing this the algorithm ensures that It will find the best strategies as for $T \rightarrow \infty$, the algorithm will have
explore all the actions in all the states and thus will know which action is the best to choose in which state. This approach ($\epsilon$-greedy) actually tackles the exploration-exploitation dilemma.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [47]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error... 2 times same line ?!!
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y] # get reward from the board
        self.board[self.x, self.y] = 0 # erase reward from board
        game_over = self.t > self.max_time # game over if more than 500 steps
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # add 0.5 of reward in some cases of the board
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        # add -1 reward in some cases of the board
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 # if both malus and bonus on the same case -> put 0

        self.board = bonus + malus # board is 0 if no reward, -0.5 if malus, 1 if positive reward
        
        # lines added (put 0 poison/cheese outside the border)
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [48]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

+ __board__: it is an 2d-array (matrix) that represents the board of the game. It is size (N, N) = (grid_size + 4, grid_size + 4) where 4 is added to the grid_size to account for a border of width 2 in each direction. The matrix __board__ contains a 0 if there is no reward a -0.5 if there is a negative reward (malus) in some position and 1 if there is a positive reward (bonus) in some other position. These rewards are randomly sample from a binomial distribution with, by default, a probability of 10% (temperature parameter) to have a malus/bonus in each case. If there is a bonus and a malus in the same case then we discard both of them and the case contains a 0.

+ __position__: position is another matrix that indicates where the player is at a certain time $t$. The matrix contains 0 everywhere except a 1 to indicate where the player is currently.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [49]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # random we don't care about s
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [50]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # restart the game for each epoch
        state = env.reset()
        
        win, lose = 0, 0
        game_over = False
        
        while not game_over:
            a = agent.learned_act(state)
            
            state, reward, game_over = env.act(a)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/14.0. Average score (-2.5)
Win/lose count 13.0/19.0. Average score (-4.25)
Win/lose count 10.0/17.0. Average score (-5.166666666666667)
Win/lose count 10.0/13.0. Average score (-4.625)
Win/lose count 8.0/16.0. Average score (-5.3)
Win/lose count 6.5/13.0. Average score (-5.5)
Win/lose count 14.0/18.0. Average score (-5.285714285714286)
Win/lose count 7.0/11.0. Average score (-5.125)
Win/lose count 10.0/15.0. Average score (-5.111111111111111)
Win/lose count 11.0/15.0. Average score (-5.0)
Win/lose count 12.0/20.0. Average score (-5.2727272727272725)
Win/lose count 14.0/18.0. Average score (-5.166666666666667)
Win/lose count 7.5/14.0. Average score (-5.269230769230769)
Win/lose count 10.0/14.0. Average score (-5.178571428571429)
Win/lose count 12.5/15.0. Average score (-5.0)
Win/lose count 7.5/10.0. Average score (-4.84375)
Win/lose count 10.5/11.0. Average score (-4.588235294117647)
Win/lose count 6.0/12.0. Average score (-4.666666666666667)
Win/lose count 15.0/15.0

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

+ I/. We have:
\begin{align*}
Q^\pi(s,a) &= E_{\pi}\left[\sum_{t = 0}^T \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
&= E_{\pi}\left[R_0 |s_{0}=s,a_{0}=a\right]
\end{align*}

Where by definition: $$R_k = \sum\limits_{t=0}^T \gamma^t r(s_{t+k}, a_{t+k})$$

Then, conditioning on the state $s'$ in which we end-up when we take action $a$ in state $s$ we have:

\begin{align*}
Q^\pi(s,a) &= \sum\limits_{s'} E_{\pi}\left[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a, s_1 = s'] P[s_1 = s' | s_0 = s, a_0 = a \right] \\
&= \sum\limits_{s'} E_{\pi}\left[r_{1}\;|\;s_{0}=s,a_{0}=a, s_1 = s'\right]P_{s,s'}^a \\
&+ \gamma \sum\limits_{s'} E_{\pi}\left[\sum_{t = 0}^T\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a, s_1 = s'\right]P_{s,s'}^a
\end{align*}

The first sum is just: $$E_{(s',a')\sim p(.|s,a)}[r(s,a)]$$
The second sum can be espanded using the conditional expectation over all the available actions and using the Markov assumption ($r_{t+1} = r(s_t, a_t)$ depends on $s_{t}$ and $a_{t}$ and not on $s_u, a_u \forall u < t$)

\begin{align*}
&\gamma \sum\limits_{s'} E_{\pi}\left[\sum_{t = 0}^T\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a, s_1 = s'\right]P_{s,s'}^a \\
&= E_{\pi}\left[\sum_{t = 0}^T\gamma^{t}r(s_{t+1},a_{t+1})| s_1 = s'\right]P_{s,s'}^a \\
&= E_{\pi}\left[R_1\;|\; s_1 = s'\right] \\
&= \sum_{a'} E_{\pi}\left[R_1\;|\; s_1 = s', a_1 =  a'\right] P[a_1 = a' | s_1 =  s'] \\
&= \sum_{a'} E_{\pi}\left[R_1\;|\; s_1 = s', a_1 =  a'\right] \pi[s', a'] \\
&= \sum_{a'} Q^{\pi}(s', a') \pi[s', a']
\end{align*}

Where the last inequality comes from the definition of $Q^{\pi}(s,a)$ and $R_k$

So finally, we have:
\begin{align*}
Q^\pi(s,a) &= E_{(s',a')\sim p(.|s,a)}[r(s,a)] + \gamma\sum_{s'}\sum_{a'} Q^{\pi}(s', a') \pi[s', a'] P_{s,s'}^a \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)] + \gamma E_{(s',a')\sim p(.|s,a)}[Q^{\pi}(s',a')]
\end{align*}

+ II/. We have:
\begin{equation*}
Q^*(s,a) = \max_{\pi} Q^{\pi}(s, a)
\end{equation*}

so by replacing into

\begin{align*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}

it comes:

\begin{align*}
Q^*(s,a) &= \max_{\pi} Q^{\pi}(s,a) = E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{\pi} Q^{\pi}(s',a')] \\
&= E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^*(s',a')]
\end{align*}

but actually only $s' \sim \pi^*(.|s,a)$, from what immediately follows (because $Q^*(s,a)$ is the best policy i.e the maximum value, i.e the policy that chooses the best action in each state, i.e, by recurrence the policy that chooses the __action__ that maximizes the next $Q^{*}(s',a')$ value):

\begin{align*}
Q^*(s,a) = \max_{\pi} Q^{\pi}(s,a) = E_{s' \sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q^{*}(s',a')]
\end{align*}

+ III/. We use a function approximation: $Q(s, a; \theta) \approx Q^{*}(s,a)$

Where $\theta$ is the parameters of a certain neural network for example.

We can train a neural network by minimizing the following loss function at each iteration (this is what we do in the next cells):

\begin{align*}
L_i(\theta_i) = E_{s' \sim \pi^*(.|s,a)}[(y_i - Q(s, a; \theta_i))^2]
\end{align*}

i.e we minimize the mean squarred-error between $y_i$ and $Q(s, a; \theta_i))$ at each iteration $i$. But what is $y_i$ here ? (recall that we don't have the ground truth target $y_i$ !)

To tackle this problem we choose $y_i$ as being the Q-value of the best action choosen one step ahead $(s', a')$ given the weights of the previous iteration $i-1$ (i.e we use the optimal Bellman-Equation that we approach by updating the weights $\theta$ of a certain neural network):

\begin{align*}
y_i = E_{s' \sim \pi^*(.|s,a)}[r(s,a)+\gamma \max_{a'} Q(s',a'; \theta_{i-1})]
\end{align*}

By doing so, we ensure that, at each iteration, our $y_i$ roughly converges towards the optimal Q-value function and that our update try to approach this newly estimate  by minimizing the mean squarred error between the next update and the previous update.

Finally we have:
\begin{align*}
\mathcal{L}(\theta_i)=E_{s' \sim \pi^*(.|s,a)}[( r(s,a)+\gamma\max_{a'}Q(s',a',\theta_{i-1})-Q(s,a,\theta_i))^{2}]
\end{align*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [51]:
import collections

class Memory(object):
    def __init__(self, max_memory=100):
        self.memory = collections.deque(maxlen=max_memory)

    def remember(self, m):
        self.memory.append(m)
            
    def random_access(self):
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [52]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [53]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    # add decaying epsilon method for question 10
    def set_epsilon(self, e):
        self.epsilon *= e # e is the dacay percentage
        
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # sample minibatch of transistions
            # (state, next_state, action, reward, game_over)
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, 0)))
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        
        model.compile("adam", "mse")
        self.model = model

In [91]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))

Epoch 000/100 | Loss 0.0022 | Win/lose count 3.5/3.0 (0.5)
Epoch 001/100 | Loss 0.0056 | Win/lose count 9.5/8.0 (1.5)
Epoch 002/100 | Loss 0.0532 | Win/lose count 5.0/2.0 (3.0)
Epoch 003/100 | Loss 0.0033 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/100 | Loss 0.0016 | Win/lose count 5.5/2.0 (3.5)
Epoch 005/100 | Loss 0.0519 | Win/lose count 6.5/6.0 (0.5)
Epoch 006/100 | Loss 0.0021 | Win/lose count 5.5/2.0 (3.5)
Epoch 007/100 | Loss 0.0018 | Win/lose count 7.0/2.0 (5.0)
Epoch 008/100 | Loss 0.0042 | Win/lose count 3.5/1.0 (2.5)
Epoch 009/100 | Loss 0.0509 | Win/lose count 4.5/1.0 (3.5)
Epoch 010/100 | Loss 0.0018 | Win/lose count 12.5/4.0 (8.5)
Epoch 011/100 | Loss 0.0030 | Win/lose count 5.5/5.0 (0.5)
Epoch 012/100 | Loss 0.0036 | Win/lose count 4.5/4.0 (0.5)
Epoch 013/100 | Loss 0.0016 | Win/lose count 3.5/3.0 (0.5)
Epoch 014/100 | Loss 0.0012 | Win/lose count 9.5/7.0 (2.5)
Epoch 015/100 | Loss 0.0015 | Win/lose count 13.5/3.0 (10.5)
Epoch 016/100 | Loss 0.0029 | Win/lose count 8.0/3.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [54]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(64, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(4))
        
        # sgd(lr=lr, decay=1e-4, momentum=0.0)
        # adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        
        model.compile("adam", "mse")
        self.model = model

In [86]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32)
train(agent,env, epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 000/100 | Loss 0.0728 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/100 | Loss 0.0036 | Win/lose count 3.0/1.0 (2.0)
Epoch 002/100 | Loss 0.0037 | Win/lose count 3.5/1.0 (2.5)
Epoch 003/100 | Loss 0.0029 | Win/lose count 2.5/3.0 (-0.5)
Epoch 004/100 | Loss 0.0043 | Win/lose count 10.5/5.0 (5.5)
Epoch 005/100 | Loss 0.0060 | Win/lose count 6.0/2.0 (4.0)
Epoch 006/100 | Loss 0.0031 | Win/lose count 7.5/4.0 (3.5)
Epoch 007/100 | Loss 0.0599 | Win/lose count 8.5/1.0 (7.5)
Epoch 008/100 | Loss 0.0044 | Win/lose count 19.0/2.0 (17.0)
Epoch 009/100 | Loss 0.0044 | Win/lose count 14.5/1.0 (13.5)
Epoch 010/100 | Loss 0.0080 | Win/lose count 7.5/3.0 (4.5)
Epoch 011/100 | Loss 0.0111 | Win/lose count 22.0/2.0 (20.0)
Epoch 012/100 | Loss 0.0019 | Win/lose count 20.5/6.0 (14.5)
Epoch 013/100 | Loss 0.0055 | Win/lose count 11.5/1.0 (10.5)
Epoch 014/100 | Loss 0.0043 | Win/lose count 9.0/3.0 (6.0)
Epoch 015/100 | Loss 0.0018 | Win/lose count 10.5/1.0 (9.5)
Epoch 016/100 | Loss 0.0042 | Win/lose cou

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [12]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 0/0. Average score (0.0)
Win/lose count 3.0/0. Average score (0.75)
Win/lose count 1.0/0. Average score (0.8)
Win/lose count 1.0/0. Average score (0.8333333333333334)
Win/lose count 2.0/0. Average score (1.0)
Win/lose count 2.5/0. Average score (1.1875)
Win/lose count 1.5/0. Average score (1.2222222222222223)
Win/lose count 2.0/0. Average score (1.3)
Win/lose count 0.5/0. Average score (1.2272727272727273)
Win/lose count 1.0/0. Average score (1.2083333333333333)
Win/lose count 1.5/0. Average score (1.2307692307692308)
Win/lose count 1.0/0. Average score (1.2142857142857142)
Win/lose count 0.5/0. Average score (1.1666666666666667)
Win/lose count 0.5/0. Average score (1.125)
Win/lose count 1.0/0. Average score (1.1176470588235294)
Win/lose count 0.5/0. Average score (1.0833333333333333)
Win/lose count 1.5/0. Average score (1.105263157894737)
Win/lose count 1.0/0. Average score (

In [ ]:
HTML(display_videos('cnn_test90.mp4'))

In [ ]:
HTML(display_videos('fc_test90.mp4'))

The algorithm tends to behave in the same manner. Yet, the convolutional neural network tends to have slightly better results on the same environment (which was expected). Yet, they have several drawbacks:

- The agent can be stuck in a empty place (place with no bonus around) due to the fact that he does get a negative reward when he is in a place with no malus/bonus.
- The agent can also stay in a place surrounding by empty cells because it does not suffer from the fact that he already visited the state he is in.
- The exploration part remains constant which encourages the agent to explore even if it is already well aware of the environment.

To tackle the __last__ issue, one idea would be to decay the epsilon at each iteration by a certain factor. To tackle the __2 first__ issues, one idea would be to add a malus (of -0.1) in each state the agent has already visited. If the agent visits this cell again, we will decrease the malus by 0.1 each time. By doing this we ensure that the agent won't visit the same states again and again and will prefer to move to unvisited states (with 0 malus) that might be near bonus cells.

These strategies are implemented below.

__Note__: One last bit that we would need to tackle is the case when a bonus cell is surrounded by several malus cell in such a way that we would need to suffer from at least one malus before getting the bonus. Here, this case is very unlikely, and even if it happens it isn't really a problem as the malus (-0.5) is less than the bonus (1.0).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [63]:
## use those samples of code:
#In train explore:
# state, reward, game_over = env.act(action, train=True)

# ## In Environment exploring:
# # You will have to change n_state to 3 because you will use one more layer!
# reward = 0
# if train:
#     reward = -self.malus_position[self.x, self.y]
# self.malus_position[self.x, self.y] = 0.1

# reward = reward + self.board[self.x, self.y]
# # 3 "feature" states instead of 2
# state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                 self.board.reshape(self.grid_size, self.grid_size,1),
#                         self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)


def train_explore(agent,env,epoch,epsilon=0.1, decay=0.99, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(epsilon)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            agent.decay_epsilon(decay)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error... 2 times same line ?!!
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
            
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]   
        
        self.board[self.x, self.y] = 0 # erase reward from board
        game_over = self.t > self.max_time # game over if more than 500 steps

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        # add 0.5 of reward in some cases of the board
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        # add -1 reward in some cases of the board
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0 # if both malus and bonus on the same case -> put 0

        self.board = bonus + malus # board is 0 if no reward, -0.5 if malus, 1 if positive reward
        
        # lines added (put 0 poison/cheese outside the border)
        self.board[:2,:] = 0
        self.board[-2:,:] = 0
        self.board[:, :2] = 0
        self.board[:, -2:] = 0

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error, 2 times same line
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [64]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=50000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore90.mp4'))

Epoch 000/100 | Loss 0.0222 | Win/lose count 14.0/41.40000000000001 (-27.400000000000013)
Epoch 001/100 | Loss 0.0210 | Win/lose count 13.5/55.9 (-42.4)
Epoch 002/100 | Loss 0.0151 | Win/lose count 18.0/27.599999999999998 (-9.599999999999998)
Epoch 003/100 | Loss 0.0098 | Win/lose count 25.0/21.900000000000002 (3.099999999999998)
Epoch 004/100 | Loss 0.0115 | Win/lose count 26.0/20.199999999999996 (5.800000000000004)
Epoch 005/100 | Loss 0.0081 | Win/lose count 21.0/20.000000000000004 (0.9999999999999964)
Epoch 006/100 | Loss 0.0583 | Win/lose count 17.5/23.00000000000001 (-5.500000000000011)
Epoch 007/100 | Loss 0.0131 | Win/lose count 24.0/19.899999999999988 (4.100000000000012)
Epoch 008/100 | Loss 0.0418 | Win/lose count 27.5/11.399999999999986 (16.100000000000016)
Epoch 009/100 | Loss 0.0140 | Win/lose count 22.0/20.399999999999984 (1.6000000000000156)
Epoch 010/100 | Loss 0.0095 | Win/lose count 20.5/12.399999999999984 (8.100000000000016)
Epoch 011/100 | Loss 0.0129 | Win/lose cou

In [66]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore90.mp4'))

Win/lose count 24.0/4.0. Average score (20.0)
Win/lose count 24.5/2.0. Average score (21.25)
Win/lose count 23.0/7.0. Average score (19.5)
Win/lose count 26.0/0. Average score (21.125)
Win/lose count 22.0/5.0. Average score (20.3)
Win/lose count 19.5/4.0. Average score (19.5)
Win/lose count 27.5/3.0. Average score (20.214285714285715)
Win/lose count 24.5/1.0. Average score (20.625)
Win/lose count 32.5/0. Average score (21.944444444444443)
Win/lose count 25.0/4.0. Average score (21.85)
Win/lose count 26.5/4.0. Average score (21.90909090909091)
Win/lose count 24.5/6.0. Average score (21.625)
Win/lose count 18.5/2.0. Average score (21.23076923076923)
Win/lose count 24.0/4.0. Average score (21.142857142857142)
Win/lose count 21.5/6.0. Average score (20.766666666666666)
Win/lose count 21.5/4.0. Average score (20.5625)
Win/lose count 27.0/0. Average score (20.941176470588236)
Win/lose count 26.5/3.0. Average score (21.083333333333332)
Win/lose count 26.5/2.0. Average score (21.26315789473684

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***